In [60]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

In [61]:
classes = ['stripey', 'sea_goldie', 'neon_damselfish']
num_classes = len(classes)
image_size = 50

In [62]:
X_train, X_test, Y_train, Y_test = np.load('./fish_aug.npy', allow_pickle=True)
X_train = X_train.astype('float') / 256
X_test  = X_test.astype('float') / 256
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test  = np_utils.to_categorical(Y_test, num_classes)
X_train

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 

In [63]:
def model_train(X, Y):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3)) # number of images
    model.add(Activation('softmax'))

    #opt = keras.optimizers.Adam()
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
                    optimizer=opt, metrics=['accuracy'])

    model.fit(X, Y, batch_size=32, epochs=80)

    # save model
    model.save('./fish_cnn_aug.h5')

    return model

In [64]:
def model_eval(model, X, Y):
    scores = model.evaluate(X, Y, verbose=1)
    print('Test loss: ', scores[0])
    print('Test Accuracy: ', scores[1])

In [65]:
model = model_train(X_train, Y_train)
model_eval(model, X_test, Y_test)

Epoch 1/100
3840/3840 [==============================] - 16s 4ms/step - loss: 0.6936 - accuracy: 0.6948
Epoch 2/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.2675 - accuracy: 0.9034
Epoch 3/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.1722 - accuracy: 0.9362
Epoch 4/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.1123 - accuracy: 0.9570
Epoch 5/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.0746 - accuracy: 0.9727
Epoch 6/100
3840/3840 [==============================] - 18s 5ms/step - loss: 0.0580 - accuracy: 0.9776
Epoch 7/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.0398 - accuracy: 0.9857
Epoch 8/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.0320 - accuracy: 0.9904
Epoch 9/100
3840/3840 [==============================] - 17s 4ms/step - loss: 0.0246 - accuracy: 0.9909
Epoch 10/100
3840/3840 [==============================] - 17s 4m